In [57]:
import pymysql
import pandas as pd
# Database connection configuration
db_config = {
    'host': '140.238.230.93',
    'user': 'kartik',
    'password': 'Kartik@24082003',
    'database': 'zabbix',
    'cursorclass': pymysql.cursors.DictCursor  # optional, returns dict rows
}

# SQL Query
sql_query = """
            SELECT
                h.name AS host,
                t.description AS trigger_name,
                e.name AS event_name,
                e.eventid,
                e.acknowledged,
                e.clock AS start_time,
                COALESCE(er.clock, UNIX_TIMESTAMP()) AS end_time,
                COALESCE(er.clock, UNIX_TIMESTAMP()) - e.clock AS duration,
                er.eventid AS recovery_eventid
            FROM hosts h
            JOIN items i ON i.hostid = h.hostid AND i.status = 0
            JOIN functions f ON f.itemid = i.itemid
            JOIN triggers t ON t.triggerid = f.triggerid AND t.status = 0
            JOIN events e ON e.objectid = t.triggerid AND e.object = 0 AND e.value = 1
            LEFT JOIN event_recovery erc ON erc.eventid = e.eventid
            LEFT JOIN events er ON er.eventid = erc.r_eventid
            WHERE 
                h.status = 0
                AND h.flags IN (0, 4)
"""

try:
    # Connect to the database
    connection = pymysql.connect(**db_config)
    with connection:
        with connection.cursor() as cursor:
            cursor.execute(sql_query)
            results = cursor.fetchall()


except Exception as e:
    print("Error:", e)


In [58]:
df = pd.DataFrame(results)
print(df)
print(df.columns.tolist())

                 host                                   trigger_name  \
0       Zabbix server                 {HOST.NAME} has been restarted   
1       Zabbix server   Utilization of http poller processes is high   
2       Zabbix server        Utilization of poller processes is high   
3       Zabbix server  Number of installed packages has been changed   
4       Zabbix server  Number of installed packages has been changed   
...               ...                                            ...   
5239    Zabbix server                           High CPU utilization   
5240    Zabbix server                           High CPU utilization   
5241    Zabbix server                           High CPU utilization   
5242  WIN-EVKHD7M05S9                Active checks are not available   
5243    Zabbix server                           High CPU utilization   

                                           event_name  eventid  acknowledged  \
0     Zabbix server has been restarted (uptime < 10m)  

In [59]:
def get_host_by_group(host_group):
    """
    Function to get the host group for a given host.
    This is a placeholder function; implement the actual logic as needed.
    """
    query = """
    SELECT 
  g.name AS group_name,
  h.host AS host_name
FROM hstgrp g
JOIN hosts_groups hg ON g.groupid = hg.groupid
JOIN hosts h ON hg.hostid = h.hostid
WHERE g.name = %s
"""
    try:
        connection = pymysql.connect(**db_config)
        with connection:
            with connection.cursor() as cursor:
                cursor.execute(query, (host_group,))
                result = cursor.fetchall()
                if result:
                    return result
    
    except Exception as e:
        print("Error retrieving host group:", e)

    # Example logic, replace with actual implementation
    return "Default Group"  # Replace with actual group retrieval logic

In [60]:
host = get_host_by_group("ASG")

# print(host)  # Print the result of the function call
print([name['host_name'] for name in host])  # Print column names
# print(host['host_name'])  # Example usage of the function

['asg-ip-140.245.29.55', 'ip-172-31-94-226.ec2.internal', 'alma', 'test-zab-kartik', 'asg-ip-122.170.27.131', 'asg-ip-103.241.45.149', 'ip-172-31-54-55.ec2.internal', 'ip-172-31-57-240.ec2.internal']


In [61]:
def get_alerts(time_from: int = None, time_to: int = None, hostname: str = None, limit: int = 100000, host_group: str = None):
    alerts = pd.DataFrame(results)
    alerts = alerts.sort_values(by='start_time', ascending=False)

    if time_from is not None:
        alerts = alerts[alerts['start_time'] >= time_from]
    if time_to is not None:
        alerts = alerts[alerts['start_time'] <= time_to]
    if hostname is not None:
        alerts = alerts[alerts['host'] == hostname]
    if host_group is not None:
        host_list = get_host_by_group(host_group)
        host_in_group = [h['host_name'] for h in host_list]
        print(f"Hosts in group {host_group}: {host_in_group}")
        alerts = alerts[alerts['host'].isin(host_in_group)]

    if limit is not None:
        alerts = alerts.head(limit)

    return alerts


In [62]:
al = get_alerts(host_group="Zabbix servers",time_from=1748443658)

Hosts in group Zabbix servers: ['Zabbix server', 'test_heena', 'test_heena_flag4', 'template_created']


In [63]:
print(al)

               host          trigger_name  \
5243  Zabbix server  High CPU utilization   
5241  Zabbix server  High CPU utilization   
5240  Zabbix server  High CPU utilization   

                                  event_name  eventid  acknowledged  \
5243  High CPU utilization (over 90% for 5m)  2203471             0   
5241  High CPU utilization (over 90% for 5m)  2203381             0   
5240  High CPU utilization (over 90% for 5m)  2203131             0   

      start_time    end_time  duration  recovery_eventid  
5243  1748446344  1748446404        60         2203480.0  
5241  1748445744  1748445804        60         2203390.0  
5240  1748443944  1748444004        60         2203140.0  
